### Invoice Generator and Tracker Project

The following code creates a small system that takes customer information, services rendered, and prices, and then generates professional invoices (PDFs). The code also keeps track of paid/unpaid invoices and creates a simple dashboard that displays total revenue and outstanding amounts. 

In [1]:
# importing necessary libraries 
import pandas as pd
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

The following lines of code create a simple database of customers and another database of services offered. 

In [2]:
customers = pd.DataFrame({
    'CustomerID' : [1, 2], 
    'Name' : ['Marvel Inc', 'DC Corp'],
    'Email' : ['avengers@marvel.com', 'dccorp@dc.com']
})

In [3]:
display(customers)

,CustomerID,Name,Email
0,1,Marvel Inc,avengers@marvel.com
1,2,DC Corp,dccorp@dc.com


In [4]:
services = pd.DataFrame({
    'ServiceID' : [101, 102],
    'Description' : ['Bookeeping', 'Tax Filing'],
    'Price' : [500, 800]
})

In [5]:
display(services)

,ServiceID,Description,Price
0,101,Bookeeping,500
1,102,Tax Filing,800


The following is a function that generates an invoice based on selected customer and services.

In [6]:
def generate_invoice(customer_id, service_ids, invoice_date=datetime.today()):
    """
    Args:
        customer_id (int): ID of the customer 
        service_ids (list): List of IDs of selected services 
        invoice_date (datetime): Date of the invoice ()

    Returns: 
        dict: Invoice details 
    """
    # Get customer information
    customer = customers[customers['CustomerID']==customer_id].iloc[0]

    # Get selected service/s
    selected_services = services[services['ServiceID'].isin(service_ids)]

    # Calculate total amount
    total = selected_services['Price'].sum()

    # Create invoice dictionary 
    invoice = {
        'CustomerID': customer_id,
        'InvoiceDate' : invoice_date.strftime("%Y-%m-%d"),
        'CustomerName': customer['Name'],
        'CustomerEmail': customer['Email'],
        'Services': ", ".join(selected_services['Description'].values),
        'Amount': total,
        'Paid': False,  # Default status is unpaid
        'PaymentDate': None  # No payment date initially
    }

    return invoice

The following code uses the **generate_invoice** function to create invoices. The invoices are stored in a list and then converted into a dataframe for eaaier management. 

In [7]:
# Initialize an empty list to store invoices
invoice_log = []

# Example: Generate two invoices
invoice1 = generate_invoice(1, [101, 102])  # Invoice for Marvel Inc for both services
invoice2 = generate_invoice(2, [101])       # Invoice for DC Corp for one service

# Add invoices to the log
invoice_log.append(invoice1)
invoice_log.append(invoice2)

# Convert invoice log into a DataFrame for easier management
invoices_df = pd.DataFrame(invoice_log)

# View Current Invoices
display(invoices_df)

,CustomerID,InvoiceDate,CustomerName,CustomerEmail,Services,Amount,Paid,PaymentDate
0,1,2025-04-26,Marvel Inc,avengers@marvel.com,"Bookeeping, Tax Filing",1300,False,None
1,2,2025-04-26,DC Corp,dccorp@dc.com,Bookeeping,500,False,None


The following code generates PDFs for each invoice.

In [8]:
def generate_invoice_pdf(invoice, filename):
    """
    Args:
        invoice (dict): Invoice details
        filename (str): File path for the PDF
    """
    # Folder path where PDFs will be saved
    folder_path = os.path.join(os.path.expanduser("~"), "Desktop", "acc_proj", "invoices")
    # Full file path
    file_path = os.path.join(folder_path, filename)
    
    # Create the PDF
    c = canvas.Canvas(file_path, pagesize=letter)
    width, height = letter

    # Add title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(200, height - 50, f"Invoice for {invoice['CustomerName']}")

    # Invoice details
    c.setFont("Helvetica", 12)
    y_position = height - 100
    c.drawString(50, y_position, f"Customer Name: {invoice['CustomerName']}")
    y_position -= 20
    c.drawString(50, y_position, f"Customer Email: {invoice['CustomerEmail']}")
    y_position -= 20
    c.drawString(50, y_position, f"Invoice Date: {invoice['InvoiceDate']}")
    y_position -= 20
    c.drawString(50, y_position, f"Services: {invoice['Services']}")
    y_position -= 20
    c.drawString(50, y_position, f"Total Amount: ${invoice['Amount']:,.2f}")
    y_position -= 20
    c.drawString(50, y_position, f"Paid: {'Yes' if invoice['Paid'] else 'No'}")
    if invoice['Paid']:
        y_position -= 20
        c.drawString(50, y_position, f"Payment Date: {invoice['PaymentDate']}")

    # Save the PDF
    c.save()

The following code generates PDFs for each invoice using the **generate_invoice_pdf** function.

In [9]:
generate_invoice_pdf(invoice1, "invoice1_marvel.pdf")
generate_invoice_pdf(invoice2, "invoice2_DC.pdf")

The following function updates the payment status of invoices as paid based on customer ID. 

In [10]:
def mark_as_paid(customer_id):
    """
    Marks only unpaid invoices as paid based on customer ID and records payment date.
    Args:
        customer_id (int): ID of the customer
    """
    unpaid_invoices = (invoices_df['CustomerID'] == customer_id) & (invoices_df['Paid'] == False)
    invoices_df.loc[unpaid_invoices, 'Paid'] = True
    invoices_df.loc[unpaid_invoices, 'PaymentDate'] = datetime.today().strftime("%Y-%m-%d")

To test the **mark_as_paid** function, update DC Corp's invoice (CustomerID = 2) as paid. Automatic color-coding is used where green marks paid invoices and red marks unpaid invoices.

In [11]:
mark_as_paid(2) # DC Corp CustomerID = 2 

# View Updated Invoices
def highlight_paid_unpaid(row):
    """
    Applies background color based on payment status.
    """
    if row['Paid']:
        return ['background-color: #d4edda'] * len(row)  # Light green
    else:
        return ['background-color: #f8d7da'] * len(row)  # Light red

# Apply the styling and display
styled_invoices = invoices_df.style.apply(highlight_paid_unpaid, axis=1)
display(styled_invoices)

,CustomerID,InvoiceDate,CustomerName,CustomerEmail,Services,Amount,Paid,PaymentDate
0,1,2025-04-26,Marvel Inc,avengers@marvel.com,"Bookeeping, Tax Filing",1300,False,None
1,2,2025-04-26,DC Corp,dccorp@dc.com,Bookeeping,500,True,2025-04-26


In the dataframe above, DC Corp's "Paid" status has updated from False to True. 

The code below displays a simple invoice summary.

In [12]:
print("="*40)
print("Invoice Summary Dashboard")
print("="*40)

# Total Revenue Collected
total_collected = invoices_df[invoices_df['Paid']]['Amount'].sum()
print(f"Total Revenue Collected: ${total_collected:,.2f}")

# Outstanding Balance
outstanding_balance = invoices_df[~invoices_df['Paid']]['Amount'].sum()
print(f"Outstanding Balance: ${outstanding_balance:,.2f}")

print("\n--- Revenue by Customer ---")

# Revenue collected by each customer
revenue_by_customer = invoices_df[invoices_df['Paid']].groupby('CustomerName')['Amount'].sum().reset_index()
revenue_by_customer['Amount'] = revenue_by_customer['Amount'].map('${:,.2f}'.format)

display(revenue_by_customer)

Invoice Summary Dashboard
Total Revenue Collected: $500.00
Outstanding Balance: $1,300.00

--- Revenue by Customer ---


,CustomerName,Amount
0,DC Corp,$500.00
